In [405]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import random
from sklearn.preprocessing import StandardScaler
import plotly.express as px


In [406]:
final_df = pd.read_csv('../data/final_V2.csv')
final_df 

,Name,Budget(USD)_Inflated,Domestic(USD)_Inflated,Domestic_Percentage,Foreign(USD)_Inflated,Foreign_Percentage,Worldwide(USD)_Inflated,Runtime(mins),Rating,Audience_Score,...,period piece,animation,teen,film adaptation,musical,history,coming of age,sports,war film,Worldwide_profit
0,10 cloverfield lane,6.076746e+06,8.760602e+07,0.654010,4.634613e+07,0.345990,1.339521e+08,103,pg-13,0.79,...,0,0,0,0,0,0,0,0,0,22.043400
1,"10,000 bc",1.441563e+08,1.301309e+08,0.351333,2.402606e+08,0.648667,3.703914e+08,109,pg-13,0.37,...,0,0,0,0,0,0,0,0,0,2.569373
2,12 rounds,2.726275e+07,1.667757e+07,0.708013,6.877890e+06,0.291987,2.355546e+07,108,pg-13,0.45,...,0,0,0,0,0,0,0,0,0,0.864016
3,12 strong,3.979050e+07,5.209113e+07,0.644274,2.876133e+07,0.355726,8.085246e+07,130,r,0.62,...,0,0,0,0,0,1,0,0,0,2.031954
4,12 years a slave,2.539260e+07,7.195247e+07,0.301875,1.663993e+08,0.698125,2.383517e+08,134,r,0.90,...,0,0,0,0,0,1,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,zoolander,4.851315e+07,7.826600e+07,0.743197,2.704388e+07,0.256803,1.053099e+08,90,pg-13,0.80,...,0,0,0,0,0,0,0,0,0,2.170749
2685,zoolander 2,6.076746e+07,3.506124e+07,0.508592,3.387664e+07,0.491408,6.893788e+07,101,pg-13,0.20,...,0,0,0,0,0,0,0,0,0,1.134454
2686,zoom,5.014200e+07,1.717625e+07,0.958658,7.407176e+05,0.041342,1.791697e+07,93,pg,0.34,...,0,0,0,0,0,0,0,0,0,0.357325
2687,zootopia,1.823024e+08,4.147601e+08,0.333340,8.294952e+08,0.666660,1.244255e+09,108,pg,0.92,...,0,1,0,0,0,0,0,0,0,6.825228


In [407]:
final_df.columns

Index(['Name', 'Budget(USD)_Inflated', 'Domestic(USD)_Inflated',
       'Domestic_Percentage', 'Foreign(USD)_Inflated', 'Foreign_Percentage',
       'Worldwide(USD)_Inflated', 'Runtime(mins)', 'Rating', 'Audience_Score',
       'Critics_Score', 'plot_summary', 'Month', 'Day', 'Year',
       'Foreign_higher', 'emotion', 'emotion_score', 'drama', 'comedy',
       'thriller', 'action', 'adventure', 'romance film', 'horror', 'fantasy',
       'mystery', 'crime', 'indie', 'biography', 'family', 'sci-fi',
       'period piece', 'animation', 'teen', 'film adaptation', 'musical',
       'history', 'coming of age', 'sports', 'war film', 'Worldwide_profit'],
      dtype='object')

## Pre-processing for OLS 

In [408]:
# Dropping columns deemed irrelevant for the model
df_ols = final_df.drop(columns=[ "Day","Name", "plot_summary", "Worldwide(USD)_Inflated", "Domestic(USD)_Inflated", "Foreign(USD)_Inflated", "Worldwide(USD)_Inflated", "Foreign_higher"])

In [409]:
df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,...,period piece,animation,teen,film adaptation,musical,history,coming of age,sports,war film,Worldwide_profit
0,6.076746e+06,0.654010,0.345990,103,pg-13,0.79,0.91,Mar,2016,fear,...,0,0,0,0,0,0,0,0,0,22.043400
1,1.441563e+08,0.351333,0.648667,109,pg-13,0.37,0.10,Mar,2008,fear,...,0,0,0,0,0,0,0,0,0,2.569373
2,2.726275e+07,0.708013,0.291987,108,pg-13,0.45,0.31,Mar,2009,anger,...,0,0,0,0,0,0,0,0,0,0.864016
3,3.979050e+07,0.644274,0.355726,130,r,0.62,0.50,Jan,2018,fear,...,0,0,0,0,0,1,0,0,0,2.031954
4,2.539260e+07,0.301875,0.698125,134,r,0.90,0.95,Oct,2013,sadness,...,0,0,0,0,0,1,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,4.851315e+07,0.743197,0.256803,90,pg-13,0.80,0.64,Sep,2001,anger,...,0,0,0,0,0,0,0,0,0,2.170749
2685,6.076746e+07,0.508592,0.491408,101,pg-13,0.20,0.22,Feb,2016,surprise,...,0,0,0,0,0,0,0,0,0,1.134454
2686,5.014200e+07,0.958658,0.041342,93,pg,0.34,0.05,Aug,2006,neutral,...,0,0,0,0,0,0,0,0,0,0.357325
2687,1.823024e+08,0.333340,0.666660,108,pg,0.92,0.98,Mar,2016,neutral,...,0,1,0,0,0,0,0,0,0,6.825228


In [410]:
# Standardizing 

columns_to_scale = ['Budget(USD)_Inflated', 'Domestic_Percentage', 'Foreign_Percentage', 'Runtime(mins)', 'Audience_Score', 'Critics_Score', 'emotion_score']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_ols[columns_to_scale])
df_ols[columns_to_scale] = scaled_data

df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,...,period piece,animation,teen,film adaptation,musical,history,coming of age,sports,war film,Worldwide_profit
0,-0.820351,0.391747,-0.391747,-0.291947,pg-13,1.012762,1.412014,Mar,2016,fear,...,0,0,0,0,0,0,0,0,0,22.043400
1,1.305928,-0.877514,0.877514,0.052120,pg-13,-1.230673,-1.539543,Mar,2008,fear,...,0,0,0,0,0,0,0,0,0,2.569373
2,-0.494109,0.618207,-0.618207,-0.005225,pg-13,-0.803352,-0.774325,Mar,2009,anger,...,0,0,0,0,0,0,0,0,0,0.864016
3,-0.301195,0.350920,-0.350920,1.256351,r,0.104705,-0.081984,Jan,2018,fear,...,0,0,0,0,0,1,0,0,0,2.031954
4,-0.522908,-1.084916,1.084916,1.485729,r,1.600328,1.557770,Oct,2013,sadness,...,0,0,0,0,0,1,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,-0.166876,0.765751,-0.765751,-1.037423,pg-13,1.066177,0.428161,Sep,2001,anger,...,0,0,0,0,0,0,0,0,0,2.170749
2685,0.021828,-0.218058,0.218058,-0.406635,pg-13,-2.138730,-1.102276,Feb,2016,surprise,...,0,0,0,0,0,0,0,0,0,1.134454
2686,-0.141793,1.669279,-1.669279,-0.865390,pg,-1.390919,-1.721738,Aug,2006,neutral,...,0,0,0,0,0,0,0,0,0,0.357325
2687,1.893336,-0.952969,0.952969,-0.005225,pg,1.707158,1.667086,Mar,2016,neutral,...,0,1,0,0,0,0,0,0,0,6.825228


In [411]:
# one-hot encoding rating column, dropping the first column in order to avoid introducing multicollinearity
df_ols = pd.get_dummies(df_ols, columns=['Rating'], drop_first=True)
df_ols = df_ols.drop(columns="Rating_nc-17") # Only 5 nc-17 movies, not considered for analysis
# one-hot encoding month column, dropping first column for same reason
df_ols = pd.get_dummies(df_ols, columns=['Month'], drop_first=True)
# same thing for the emotion column
df_ols = pd.get_dummies(df_ols, columns=['emotion'], drop_first=True)


In [412]:
df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Audience_Score,Critics_Score,Year,emotion_score,drama,comedy,...,Month_May,Month_Nov,Month_Oct,Month_Sep,emotion_disgust,emotion_fear,emotion_joy,emotion_neutral,emotion_sadness,emotion_surprise
0,-0.820351,0.391747,-0.391747,-0.291947,1.012762,1.412014,2016,0.050657,1,0,...,False,False,False,False,False,True,False,False,False,False
1,1.305928,-0.877514,0.877514,0.052120,-1.230673,-1.539543,2008,0.504088,1,0,...,False,False,False,False,False,True,False,False,False,False
2,-0.494109,0.618207,-0.618207,-0.005225,-0.803352,-0.774325,2009,0.056938,0,0,...,False,False,False,False,False,False,False,False,False,False
3,-0.301195,0.350920,-0.350920,1.256351,0.104705,-0.081984,2018,1.226752,1,0,...,False,False,False,False,False,True,False,False,False,False
4,-0.522908,-1.084916,1.084916,1.485729,1.600328,1.557770,2013,0.964755,1,0,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,-0.166876,0.765751,-0.765751,-1.037423,1.066177,0.428161,2001,-0.726523,0,1,...,False,False,False,True,False,False,False,False,False,False
2685,0.021828,-0.218058,0.218058,-0.406635,-2.138730,-1.102276,2016,1.495326,0,1,...,False,False,False,False,False,False,False,False,False,True
2686,-0.141793,1.669279,-1.669279,-0.865390,-1.390919,-1.721738,2006,0.055972,0,0,...,False,False,False,False,False,False,False,True,False,False
2687,1.893336,-0.952969,0.952969,-0.005225,1.707158,1.667086,2016,0.256964,0,1,...,False,False,False,False,False,False,False,True,False,False


## Defining features and labels for the 4 different models:
Labels: Foreign/domestic percentage for prediction
Predictors: before and after 2010

In [413]:
# Removing Year column, not used as a feature

# Movies before 2010
df_before_2010 = df_ols[df_ols['Year'] < 2010].drop(columns= "Year")

# Movies including and after 2010
df_2010_and_after = df_ols[df_ols['Year'] >= 2010].drop(columns= "Year")

df_ols = df_ols.drop(columns= "Year")


Function to train OLS model

In [414]:
def train_ols_model(dataframe, target_column, drop_columns):
    """
    Trains an OLS regression model.

    Parameters:
        dataframe (pd.DataFrame): The input DataFrame containing features and the target variable.
        target_column (str): The name of the column to be used as the target variable.
        drop_columns (list): List of columns to drop from the DataFrame.

    Returns:
        ols_model (sm.OLS): The trained OLS model.
    """
    # Defining features and labels
    y = dataframe[target_column]
    X = dataframe.drop(columns=drop_columns)

    # Adding a constant column for the OLS model
    X = sm.add_constant(X)

    # Training the OLS model
    feature_names = list(X.columns)
    label_name = target_column

    # Ensure predictors and target are numeric
    X = pd.DataFrame(X, columns=feature_names, dtype=float)
    y = pd.Series(y, name=label_name, dtype=float) # Including the label name

    # Fit the OLS model
    ols_model = sm.OLS(y, X).fit()
    return ols_model

Function to save the feature statistics for later use

In [415]:
def extract_feature_statistics(ols_model, output_csv_path=None):
    """
    Extracts feature statistics from an OLS regression model.

    Parameters:
        ols_model (sm.OLS): The trained OLS regression model.
        output_csv_path (str, optional): Path to save the feature statistics as a CSV file. Defaults to None.

    Returns:
        feature_stats (pd.DataFrame): A DataFrame containing the feature statistics.
    """
    # Extracting feature statistics
    coefficients = ols_model.params  # Coefficients (coef)
    standard_errors = ols_model.bse  # Standard errors (std err)
    t_values = ols_model.tvalues     # t-values (t)
    p_values = ols_model.pvalues     # p-values (P>|t|)

    # Creating a DataFrame
    feature_stats = pd.DataFrame({
        "Feature": coefficients.index,
        "Coefficient": coefficients.values,
        "Std_Error": standard_errors.values,
        "t_value": t_values.values,
        "p_value": p_values.values
    })

    # Saving the DataFrame to a CSV file if the path is provided
    if output_csv_path:
        feature_stats.to_csv(output_csv_path, index=False)

    return feature_stats

### 1st model: Foreign Percentage prediction 

In [416]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_Foreign.csv"

# Train the model
ols_model = train_ols_model(df_ols, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_Foreign = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.233
Method:                 Least Squares   F-statistic:                     17.65
Date:                Fri, 20 Dec 2024   Prob (F-statistic):          4.32e-127
Time:                        15:26:38   Log-Likelihood:                -3434.4
No. Observations:                2689   AIC:                             6969.
Df Residuals:                    2639   BIC:                             7264.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.2354 

### "2nd" model: Domestic Percentage prediction 

In [417]:
# Defining the target column as well as which columns to drop
target_column = "Domestic_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_Domestic.csv"

# Train the model
ols_model = train_ols_model(df_ols, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_domestic = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                             OLS Regression Results                            
Dep. Variable:     Domestic_Percentage   R-squared:                       0.247
Model:                             OLS   Adj. R-squared:                  0.233
Method:                  Least Squares   F-statistic:                     17.65
Date:                 Fri, 20 Dec 2024   Prob (F-statistic):          4.32e-127
Time:                         15:26:38   Log-Likelihood:                -3434.4
No. Observations:                 2689   AIC:                             6969.
Df Residuals:                     2639   BIC:                             7264.
Df Model:                           49                                         
Covariance Type:             nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 

Only difference between the two models is the sign of the coeffecients. Subsquent Analysis will have only the foreign_percentage as a target column

### Before 2010, foreign prediction

In [418]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_pre_2010.csv"

# Train the model
ols_model = train_ols_model(df_before_2010, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_pre_2010 = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.233
Method:                 Least Squares   F-statistic:                     8.619
Date:                Fri, 20 Dec 2024   Prob (F-statistic):           1.81e-50
Time:                        15:26:38   Log-Likelihood:                -1480.0
No. Observations:                1232   AIC:                             3060.
Df Residuals:                    1182   BIC:                             3316.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.2585 

### Including and after 2010, foreign prediction

In [419]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_post_2010.csv"

# Train the model
ols_model = train_ols_model(df_2010_and_after, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_post_2010 = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     10.62
Date:                Fri, 20 Dec 2024   Prob (F-statistic):           3.44e-66
Time:                        15:26:38   Log-Likelihood:                -1869.0
No. Observations:                1457   AIC:                             3838.
Df Residuals:                    1407   BIC:                             4102.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.3810 

## Comparing the statistically significant coefficients

In [420]:
feature_stats_pre_2010.head(5)

,Feature,Coefficient,Std_Error,t_value,p_value
0,const,-0.258459,0.214876,-1.202826,2.292845e-01
1,Budget(USD)_Inflated,0.216120,0.038300,5.642805,2.092620e-08
2,Runtime(mins),0.009150,0.033127,0.276207,7.824370e-01
3,Audience_Score,-0.078971,0.034722,-2.274359,2.312303e-02
4,Critics_Score,0.140178,0.034487,4.064683,5.127671e-05


In [421]:
feature_stats_post_2010.head(5)

,Feature,Coefficient,Std_Error,t_value,p_value
0,const,-0.380963,0.372023,-1.024031,3.059966e-01
1,Budget(USD)_Inflated,0.271148,0.035662,7.603285,5.250117e-14
2,Runtime(mins),0.084395,0.033909,2.488883,1.292940e-02
3,Audience_Score,-0.221952,0.036483,-6.083684,1.511206e-09
4,Critics_Score,0.069127,0.035091,1.969956,4.903930e-02


Defining funciton that removes all the features with a P-value > 0.05

In [422]:
def filter_significant_features(feature_stats, p_value_threshold=0.05):

    # Filter the DataFrame based on the p-value threshold
    significant_features = feature_stats[feature_stats["p_value"] <= p_value_threshold]
    
    return significant_features

In [423]:
feature_stats_pre_2010 = filter_significant_features(feature_stats_pre_2010)
feature_stats_post_2010 = filter_significant_features(feature_stats_post_2010)

In [424]:
feature_stats_pre_2010

,Feature,Coefficient,Std_Error,t_value,p_value
1,Budget(USD)_Inflated,0.216120,0.038300,5.642805,2.092620e-08
3,Audience_Score,-0.078971,0.034722,-2.274359,2.312303e-02
4,Critics_Score,0.140178,0.034487,4.064683,5.127671e-05
7,comedy,-0.231626,0.060537,-3.826157,1.369663e-04
11,romance film,0.185012,0.062931,2.939938,3.346622e-03
13,fantasy,0.192748,0.085760,2.247533,2.478979e-02
14,mystery,0.188412,0.092842,2.029392,4.264210e-02
21,animation,0.421005,0.145252,2.898447,3.819396e-03
27,sports,-0.481573,0.106367,-4.527477,6.572641e-06
28,war film,0.309143,0.128523,2.405358,1.630953e-02


In [425]:
feature_stats_post_2010

,Feature,Coefficient,Std_Error,t_value,p_value
1,Budget(USD)_Inflated,0.271148,0.035662,7.603285,5.250117e-14
2,Runtime(mins),0.084395,0.033909,2.488883,1.292940e-02
3,Audience_Score,-0.221952,0.036483,-6.083684,1.511206e-09
4,Critics_Score,0.069127,0.035091,1.969956,4.903930e-02
5,emotion_score,0.069881,0.024964,2.799239,5.192268e-03
7,comedy,-0.225915,0.061611,-3.666828,2.547237e-04
8,thriller,0.199663,0.067346,2.964726,3.080530e-03
9,action,0.166445,0.070575,2.358427,1.848869e-02
10,adventure,0.304298,0.075784,4.015345,6.249093e-05
21,animation,0.539948,0.148577,3.634124,2.889661e-04


Plotting the coeffecients for the statistically significant features

In [426]:
import plotly.express as px

df = feature_stats_pre_2010
fig = px.bar(
    df, 
    x='Feature', 
    y='Coefficient',
    hover_data=['Std_Error', 't_value', 'p_value'], 
    color='Coefficient',
    labels={'Feature': 'Feature Names', 'Coefficient': 'Coefficient Value'},  # Correctly map labels
    height=400
)
fig.show()
fig.write_html("../_includes/OLS_features_pre_2010.html")

In [427]:
import plotly.express as px

# Your dataset
df = feature_stats_post_2010

# Sorting features by absolute coefficient value for better visualization (optional)
df_sorted = df.sort_values(by="Coefficient", key=abs, ascending=False)

# Creating the bar plot
fig = px.bar(
    df_sorted, 
    x='Feature', 
    y='Coefficient', 
    hover_data=['Std_Error', 't_value', 'p_value'], 
    color='Coefficient',
    labels={'Feature': 'Feature Names', 'Coefficient': 'Coefficient Value'},  # Map column names for display
    height=400
)

# Display the plot
fig.show()
fig.write_html("../_includes/OLS_features_post_2010.html")